In [27]:
import pandas as pd
import json

# CSV 파일 경로 설정
csv_file_path = "/home/do/Downloads/mid360-real-centr.csv"

# CSV 파일 불러오기
df = pd.read_csv(csv_file_path)

# 열 이름이 정확히 맞지 않으면 아래처럼 수정해줘야 합니다.
# df.columns = ["Time/s", "Azimuth/deg", "Zenith/deg"]

# Zenith → Elevation 변환 (Elevation = 90 - Zenith)
df["Elevation/deg"] = 90 - df["Zenith/deg"]

# 상위 200000개 데이터만 사용
num_sample = 20800*5
df_sampled = df.head(num_sample)
df_sampled = df_sampled[20800*4:]
# 4개마다 하나씩 총 20만개 사용
# df_sampled = df.iloc[::4].reset_index(drop=True)

# emitterStates 구성
emitter_state = {
    "azimuthDeg": df_sampled["Azimuth/deg"].round(6).tolist(),
    "elevationDeg": df_sampled["Elevation/deg"].round(6).tolist(),
    "fireTimeNs": [i * 1.6 for i in range(len(df_sampled))],
    "channelId": list(range(len(df_sampled))),
    "rangeId": [0] * len(df_sampled),
    "bank": list(reversed(range(len(df_sampled))))
}

# JSON config 전체 구성
lidar_config = {
    "class": "sensor",
    "type": "lidar",
    "name": "Simple Solid State",
    "driveWorksId": "GENERIC",
    "profile": {
        "scanType": "solidState",
        "intensityProcessing": "normalization",
        "rotationDirection": "CW",
        "rayType": "IDEALIZED",
        "nearRangeM": 0.1,
        "farRangeM": 100.0,
        "effectiveApertureSize": 0.0145,
        "focusDistM": 0.16,
        "rangeResolutionM": 0.003,
        "rangeAccuracyM": 0.02,
        "avgPowerW": 0.002,
        "minReflectance": 0.1,
        "minReflectanceRange": 70.0,
        "wavelengthNm": 905.0,
        "pulseTimeNs": 6,
        "maxReturns": 2,
        "scanRateBaseHz": 10.0,
        "reportRateBaseHz": 30,
        "numberOfEmitters": 20800,
        "numberOfChannels": 20800,
        "rangeCount": 1,
        "ranges": [{"min": 0.5, "max": 300}],
        "azimuthErrorMean": 0.0,
        "azimuthErrorStd": 0.05,
        "elevationErrorMean": 0.0,
        "elevationErrorStd": 0.05,
        "stateResolutionStep": 1,
        "numLines": 1,
        "numRaysPerLine": [20800],
        "emitterStateCount": 1,
        "emitterStates": [emitter_state],
        "intensityMappingType": "LINEAR"
    }
}

# JSON 파일로 저장
output_path = "/home/do/Desktop/lidar_configs/Livox/MID360_5.json"
with open(output_path, "w") as f:
    json.dump(lidar_config, f, indent=2)

print(f"✅ JSON 파일 저장 완료: {output_path}")


✅ JSON 파일 저장 완료: /home/do/isaacsim/exts/isaacsim.sensors.rtx/data/lidar_configs/Livox/test_livox6.json


In [ ]:
from pxr import Usd, Gf, Sdf
import omni.usd

stage = omni.usd.get_context().get_stage()

lidar_prim_path = "/fuel/turtlebot3_burger/base_footprint/base_link/sensor_pack/sensors_link/lidar_link/Lidar"
lidar_prim = stage.GetPrimAtPath(lidar_prim_path)


attrs = lidar_prim.GetAttributes()

for attr in attrs:
    print(attr.GetName())
sensor_config_attr = lidar_prim.GetAttribute("sensorModelConfig")
config = sensor_config_attr.Get()
print(config)
# azimuth_attr = lidar_prim.GetAttribute("profile:emitterStates:0:elevationDeg")

# current_azimuth = azimuth_attr.Get()
# print("azimuthDeg:", current_azimuth)
# if not azimuth_attr.IsValid():
#     print("Invalid attr")

# else:
#     new_azimuth = [
#             -2, -1, 1, 2,
#             -1.5, -0.5, 0.5, 1.5,
#             -2.4, -1.4, 1.4, 2.4
#             ]
#     for _ in range(len(new_azimuth)):
#         new_azimuth[_] /= 10 
#     azimuth_attr.Set(new_azimuth)